<a href="https://colab.research.google.com/github/Gail529/Mental-health/blob/main/streamlit_MH_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install streamlit
!pip install pyngrok==4.1.1
!pip install ngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15985 sha256=8b1e617316aae8aad47f4f04184dccf28940f7489d0851caf42b1c1fb94ae9c6
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [1]:
import streamlit as st
import pickle

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd 
import numpy as np
import gensim
import os
import tweepy as tw
 
#tweet preprocessing 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from nltk.tokenize import word_tokenize,sent_tokenize,TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
 
 
 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional

2021-06-16 04:11:22.517 INFO    summarizer.preprocessing.cleaner: 'pattern' package not found; tag filters are not available for English


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
#lowercasing and url,punctuations and numbers removal,
def Lowercasing(words):
    string=re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "",str(words))
    word=string.lower()
    return word

#Tokenization and (@)handle extraction
def Tokenization(tweet):
    tokenizer=TweetTokenizer(strip_handles=True)
    tokens = tokenizer.tokenize(tweet)
    return tokens

#punctuations
def Punctuation_removal(tokens):
    words=[ word for word in tokens if word.isalnum()]
    return words

#stemming
def stemming(text):
    stemmer=PorterStemmer()
    for  word in text:
        stemmed_words=stemmer.stem(word)
        return stemmed_words

#stopword_removal
def remove_stopwords(words):
    stop_words=set(stopwords.words("english")) 
    result=[word for word in words if word not in stop_words ]
    return result


#lemmatization
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    lemmatized_phrase=[]
    for word in text:
        lemmatized_word=lemmatizer.lemmatize(word)
        lemmatized_phrase.append(lemmatized_word)
    return lemmatized_phrase
 


def clean_tweet(tweet):
    tweet_tokens=Tokenization(tweet)
    lemmatized_tweet=lemmatization(tweet_tokens)#lemmatization
    lowercased_string=Lowercasing(lemmatized_tweet)#lowercasing and removing numbers
    return lowercased_string


In [21]:
%%writefile app.py
import streamlit as st
import pickle

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd 
import numpy as np
import gensim
import os
import tweepy as tw
 
#tweet preprocessing 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from nltk.tokenize import word_tokenize,sent_tokenize,TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
 
 
 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import keras
import tensorflow as tf


def Lowercasing(words):
    string=re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "",str(words))
    word=string.lower()
    return word

#Tokenization and (@)handle extraction
def Tokenization(tweet):
    tokenizer=TweetTokenizer(strip_handles=True)
    tokens = tokenizer.tokenize(tweet)
    return tokens

#punctuations
def Punctuation_removal(tokens):
    words=[ word for word in tokens if word.isalnum()]
    return words

#stemming
def stemming(text):
    stemmer=PorterStemmer()
    for  word in text:
        stemmed_words=stemmer.stem(word)
        return stemmed_words

#stopword_removal
def remove_stopwords(words):
    stop_words=set(stopwords.words("english")) 
    result=[word for word in words if word not in stop_words ]
    return result


#lemmatization
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    lemmatized_phrase=[]
    for word in text:
        lemmatized_word=lemmatizer.lemmatize(word)
        lemmatized_phrase.append(lemmatized_word)
    return lemmatized_phrase
 


def clean_tweet(tweet):
    tweet_tokens=Tokenization(tweet)
    lemmatized_tweet=lemmatization(tweet_tokens)#lemmatization
    lowercased_string=Lowercasing(lemmatized_tweet)#lowercasing and removing numbers
    return lowercased_string


def predict(message):
    model=load_model('/content/MentalHealth_model.h5')
    with open('/content/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    data = clean_tweet(message)
    seq=tokenizer.texts_to_sequences(data)
    data = pad_sequences(seq,maxlen=200)
    prediction=model.predict(data)
    output = prediction.argmax(axis=1)[0][0]
    return output

 
st.title('Depressive tweets detector')
message = st.text_area('Enter/paste Tweet,”Type Here ..')
if st.button('Analyze'):
    with st.spinner('Analyzing the text …'):
        prediction=predict(message)
        if prediction == 0:
            st.success('Most likely to be depressed')
        if prediction == 1:
            st.success('quite close to being depressed')
        if prediction == 2:
            st.success('neutral')
        if prediction == 3:
            st.success('not likely to be depressed')
        if prediction == 4:
            st.success('definitely not  depressed')

Overwriting app.py


In [9]:
! ngrok authtoken 1n3n9zfRI524JaNlhCuS3GHARgM_7Z465eCAfxd5tsaavs7Tj


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
! ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [11]:
!streamlit run app.py &>/dev/null&

In [12]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url


2021-06-16 04:35:01.787 INFO    pyngrok.process: ngrok process starting: 451
2021-06-16 04:35:01.808 INFO    pyngrok.process: t=2021-06-16T04:35:01+0000 lvl=info msg="no configuration paths supplied"

2021-06-16 04:35:01.810 INFO    pyngrok.process: t=2021-06-16T04:35:01+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2021-06-16 04:35:01.818 INFO    pyngrok.process: t=2021-06-16T04:35:01+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2021-06-16 04:35:01.824 INFO    pyngrok.process: t=2021-06-16T04:35:01+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2021-06-16 04:35:01.983 INFO    pyngrok.process: t=2021-06-16T04:35:01+0000 lvl=info msg="tunnel session started" obj=tunnels.session

2021-06-16 04:35:01.985 INFO    pyngrok.process: t=2021-06-16T04:35:01+0000 lvl=info msg="client session established" obj=csess id=3938e892f1a2

2021-06-16 04:35:01.996 INFO    pyngrok.process: ngrok process has s

'http://634c9188509e.ngrok.io'

# Streamlit code


In [ ]:
!pip install streamlit

In [ ]:
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

wordnet = WordNetLemmatizer()
regex = re.compile('[%s]' % re.escape(string.punctuation))


In [ ]:
MODEL_PATH = '/content/MH/saved_model.pb'
MAX_NB_WORDS = 100000 # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 200 # max length of each entry (sentence), including padding
VALIDATION_SPLIT = 0.2 # data for validation (not used in training)
EMBEDDING_DIM = 100
tokenizer_file = 'tokenizer.pickle'

In [ ]:
with open(tokenizer_file, 'rb') as handle:
   tokenizer = pickle.load(handle)
@st.cache(allow_output_mutation=True)
def Load_model():
   model = reconstructed_model
   model.summary() # included to make it visible when model is reloaded
   session = K.get_session()
   return model, session

   Load_model

In [ ]:
#@st.cache
#def load_my_model():
model = reconstructed_model
 

if __name__ == '__main__':
    st.title('A simple streamlit mental health app')
    st.write('Want to know if your tweets contain some depressive characteristics')
    st.subheader('Key in or paste one of your tweets') 
    tweet=st.text_area('') 
    prediction_btn = st.button('predict')
    if prediction_btn:
        clean_text = []
        #K.set_session(session)
        i = clean_tweet(tweet)
        clean_text.append(i)
        sequences = tokenizer.texts_to_sequences(clean_text)
        data = pad_sequences(sequences, maxlen = max_length)
        prediction = model.predict(data)

In [ ]:
if prediction_btn:
   clean_text = []
   K.set_session(session)
   i = clean_tweet(tweet)
   clean_text.append(i)
   sequences = tokenizer.texts_to_sequences(clean_text)
   data = pad_sequences(sequences, maxlen = max_length)
   prediction = model.predict(data)

In [ ]:
prediction_class = prediction.argmax(axis=-1)[0]


st.header("Prediction using LSTM model")
if prediction_class == 0:
    st.success()
if prediction_class == 1:
    st.success()
if prediction_class == 2:
    st.success()
if prediction_class == 3:
    st.success()
if prediction_class == 4:
    st.success()

In [ ]:
if __name__ == '__main__':
    st.title('A simple streamlit mental health app')
    st.write('Want to know if your tweets contain some depressive characteristics')
    st.subheader('Key in or paste one of your tweets') 
    tweet=st.text_area('') 
    prediction_btn = st.button('predict')
    model= Load_model()
